In [1]:
#from tensorflow.python import keras
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

#from keras import backend
#backend.set_session(session)
#print(backend._get_available_gpus())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18049475020248702611
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12725320369157067470
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1191023487162741697
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5324912736
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18303899297787449109
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7

'2.3.0'

In [2]:

import numpy
import tensorflow
import keras

import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Input, Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding
from keras.preprocessing.sequence import pad_sequences

from random import randint, choice
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

import pickle
import random

#import nltk
#nltk.download('averaged_perceptron_tagger')

import sys
sys.path.insert(1, '../utils')
from NLP_little_helpers import *
#from attention_model import *
little_helpers()

[nltk_data] Error loading brown: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, clean_entire, predict_tags, tagged_3D, tagged_n_grams, unknown_words_X, check_and_predict


In [3]:
def attention_model(n_timesteps_in, n_features):
    model =  keras.Sequential()
    model.add(LSTM(8, input_shape=(n_timesteps_in, n_features), return_sequences=True))
    model.add(AttentionDecoder(8, n_features))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model


# train and evaluate a model, return accuracy
def train_model(X_in, y_in, model, epochs, batch_size, vocab_size):
    # train LSTM   
    for epoch in range(epochs):
        b = 0
        # train on mini-batches
        for batch in range(int(len(X_in)/batch_size)):
            X = X_in[b:b+batch_size]
            y = y_in[b:b+batch_size]
            b += batch_size
            
            # one hot encode X and y
            X_enc = [tokenizer.texts_to_matrix(x) for x in X]
            y_enc = [tokenizer.texts_to_matrix(i) for i in y]
            X_pad = pad_sequences(X_enc, maxlen=max_length, padding='post', truncating='post')
            y_pad = pad_sequences(y_enc, maxlen=max_length, padding='post', truncating='post')

            # fit model for one epoch on this sequence
            model.fit(X_pad, y_pad, batch_size=2, epochs=1, verbose=int(len(X_in)/batch_size))
            
            
# train and evaluate a model, return accuracy
def generator(X_in, y_in, batch_size, vocab_size):
    # train LSTM   
    b = 0
    
    while True:
        X = X_in[b:b+batch_size]
        y = y_in[b:b+batch_size]
        b += batch_size

        # one hot encode X and y
        X_enc = [tokenizer.texts_to_matrix(x) for x in X]
        y_enc = [tokenizer.texts_to_matrix(i) for i in y]
        X_pad = pad_sequences(X_enc, maxlen=max_length, padding='post', truncating='post')
        y_pad = pad_sequences(y_enc, maxlen=max_length, padding='post', truncating='post')

        yield X_pad, y_pad

            
def evaluate_model(X_pad, y_pad, model, vocab_size):
    # evaluate LSTM
    total, correct = len(X_pad), 0
    for i in range(total):
        X_enc = to_categorical([X_pad[i]], num_classes=vocab_size)
        yhat = model.predict(X_enc, verbose=0)[0]
        dec = [np.argmax(i) for i in yhat]
        if dec == X_pad[i]:
            correct += 1
            
    return float(correct)/float(total)*100.0
        

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]


# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices, word_index):
    # Looking up words in dictionary
    words = [word_index[word] for word in list_of_indices if word != 0]
    return words

In [4]:
# Load the data
lines = open(r'../Data/cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open(r'../Data/cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        q = ['<bos>'] + ' , '.join(clean_entire(id2line[conv[i]])).split() + ['<eos>']
        a = ['<bos>'] + ' , '.join(clean_entire(id2line[conv[i+1]])).split() + ['<eos>']
        if len(q) <= 80 and len(q) > 5 and len(a) <= 80 and len(a) > 5:
            questions.append(q)   
            answers.append(a)

# Check if we have loaded the data correctly
print(questions[0])
print(answers[0])
print()
print(len(questions), len(answers))

['<bos>', 'can', 'we', 'make', 'this', 'quick', ',', 'roxanne', 'korrine', 'and', 'andrew', 'barrett', 'are', 'having', 'an', 'incredibly', 'horrendous', 'public', 'break', 'up', 'on', 'the', 'quad', ',', 'again', '<eos>']
['<bos>', 'well', ',', 'i', 'thought', 'we', 'would', 'start', 'with', 'pronunciation', ',', 'if', 'that', 'is', 'okay', 'with', 'you', '<eos>']

141188 141188


In [5]:
max_length = max([len(sent) for sent in questions+answers])
vocab_size = 6000

# integer encode sequences of words
tokenizer = Tokenizer(filters='', oov_token=1, num_words=vocab_size)
tokenizer.fit_on_texts(questions[:20000]+answers[:20000])
tokenizer.word_index[0] = '<pad>'

# create weights to avoid high frequent words
all_tokens = [j for i in tokenizer.texts_to_sequences(questions[:20000]+answers[:20000]) for j in i]
class_weights = class_weight.compute_class_weight('balanced', np.unique(all_tokens), all_tokens)

X_train, X_test, y_train, y_test = train_test_split(questions[:20000], answers[:20000], test_size=0.2, random_state=42)

/home/felipe/anaconda3/envs/data-sci/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[   1    2    3 ... 5997 5998 5999], y=[3, 54, 22, 104, 25, 1011, 2, 1, 1, 15, 1, 2428, 17, 381, 77, 4036, 1, 881, 460, 57, 33, 7, 1, 2, 183, 4, 3, 60, 2, 6, 137, 22, 38, 262, 36, 1, 2, 43, 14, 9, 116, 36, 5, 4, 3, 11, 7, 4037, 15, 1, 15, 1, 348, 2, 145, 4, 3, 7, 118, 9, 2, 5277, 6, 23, 56, 7, 1854, 16, 10, 2753, 1, 1, 16, 2754, 2, 29, 715, 2, 6, 73, 550, 316, 63, 100, 4, 3, 1, 2982, 2, 63, 278, 8, 32, 97, 2135, 80, 63, 540, 506, 320, 2, 86, 12, 31, 37, 40, 63, 58, 428, 16, 12, 78, 95, 4, 3, 4038, 2, 43, 114, 22, 70, 143, 3271, 10, 913, 4, 3, 1, 1326, 1, 2, 25, 9, 29, 317, 4, 3, 64, 2, 68, 2, 5, 17, 472, 27, 7, 1, 4, 3, 6, 13, 11, 52, 8, 30, 53, 8, 84, 14, 471, 2, 6, 52, 8, 30, 2281, 158, 2, 40, 89, 7, 74, 4587, 17, 2, 53, 121, 100, 2579, 914, 2, 259, 40, 2755, 2, 6, 18, 93, 19, 29, 140, 99, 8, 304, 49, 29, 317, 8, 235, 4, 3, 53, 9, 115, 110, 143, 7, 1, 10, 550, 

In [ ]:
model = attention_model(max_length, vocab_size)

In [ ]:
epochs = 1
batch_size = 4

gen = generator(X_train, y_train, batch_size, vocab_size+1)
model.fit_generator(gen, steps_per_epoch=len(X_train)/batch_size, epochs=2, class_weight=class_weights)

In [ ]:
from keras.optimizers import Adam

In [ ]:
tf.__version__

In [ ]:
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True, dropout=0.25)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True, dropout=0.25)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model


# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        if np.argmax(yhat) == 3:
            break
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequencebatch_size=
        target_seq = yhat
    return np.array(output)


 
def generator_plain(X1, X2, y_, batch_size): # Create empty arrays to contain batch of features and labels# batch_features = np.zeros((batch_size, 64, 64, 3))
    
    X_enc = np.zeros((batch_size, 27, 4287))
    X_dec = np.zeros((batch_size, 29, 4287))
    y = np.zeros((batch_size, 29, 4287))
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index=random.choice(range(len(X1)))
            X_enc[i] = X1[index]
            X_dec[i] = X2[index]
            y[i] = y_[index]
        yield [X_enc, X_dec], y
        
        
                
def one_hot_encode(sequence, word_index):
    
    gabarito = np.zeros(len(word_index))
    gab = np.array([gabarito] * len(sequence))

    for i in range(len(sequence)):
        for j, v in enumerate(word_index.values()):
            if sequence[i] == v:
                gab[i][j] = 1.
                
    return gab
        
        
def generator_weird(X1, X2, y, batch_size, vocab_enc, vocab_dec, Q_word_index, A_word_index): 
    
    enc = np.zeros((batch_size, X1.shape[1], vocab_enc))
    dec = np.zeros((batch_size, X2.shape[1], vocab_dec))
    target = np.zeros((batch_size, y.shape[1], vocab_dec))
    print('enc', enc.shape)
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = random.choice(range(len(X1)))
            print(index)
            print(X1[index].shape)
            enc[i] = one_hot_encode(X1[index], Q_word_index)
            dec[i] = one_hot_encode(X2[index], A_word_index)
            target[i] = one_hot_encode(y[index], A_word_index)
        print(enc)

        yield [enc, dec], target
        
        
def generator(X1, X2, y, batch_size, vocab_enc, vocab_dec, Q_word_index, A_word_index): 
    
    while True:
        enc = np.array(list(map(lambda x: to_categorical(x, num_classes=len(t_Q.word_index)+2), X1)))
        dec = np.array(list(map(lambda x: to_categorical(x, num_classes=len(t_A.word_index)+2), X2)))
        target = np.array(list(map(lambda x: to_categorical(x, num_classes=len(t_A.word_index)+2), y)))

        yield [enc, dec], target
        
        

          

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]


# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices, word_index):
    # Looking up words in dictionary
    words = [word_index[word] for word in list_of_indices if word != 0]
    return words

In [ ]:
# Load the data
lines = open(r'../Data/cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open(r'../Data/cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [ ]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        q = ['<bos>'] + ' , '.join(clean_entire(id2line[conv[i]])).split() + ['<eos>']
        a = ['<bos>'] + ' , '.join(clean_entire(id2line[conv[i+1]])).split() + ['<eos>']
        if len(q) <= 200 and len(q) > 5 and len(a) <= 200 and len(a) > 5:
            questions.append(q)   
            answers.append(a)

# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

print(len(questions))
print(len(answers))

In [ ]:
Q_max = max([len(sent) for sent in questions])
A_max = max([len(sent) for sent in answers])

# integer encode sequences of words
t_Q = Tokenizer(filters='', oov_token=1)
t_A = Tokenizer(filters='', oov_token=1)

t_Q.word_index[0] = '<pad>'
t_A.word_index[0] = '<pad>'

t_Q.fit_on_texts(questions)
t_A.fit_on_texts(answers)

vocab_Q = len(t_Q.word_index.items())+2
vocab_A = len(t_A.word_index.items())+2

print('Vocabulary Size: ', vocab_Q, vocab_A)
print('Max length:', Q_max, A_max, '\n')
          
Q_enc = t_Q.texts_to_sequences(questions)
A_enc = t_A.texts_to_sequences(answers)

print(Q_enc[0])
print(A_enc[0])

In [ ]:
enc = [Q[1:-1] for Q in Q_enc]
dec = [A[:-1] for A in A_enc]
tar = [A[1:] for A in A_enc]

X_enc = pad_sequences(enc, maxlen=Q_max, padding='post', truncating='post')
X_dec = pad_sequences(dec, maxlen=A_max, padding='post', truncating='post')
y = pad_sequences(tar, maxlen=A_max, padding='post', truncating='post')

# create weights to avoid high frequent words
all_tokens = [j for i in Q_enc+A_enc for j in i]
class_weights = class_weight.compute_class_weight('balanced', np.unique(all_tokens), all_tokens)

In [ ]:
# define model
train, infenc, infdec = define_models(vocab_Q, vocab_A, 1)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
training_generator = generator(X_enc, X_dec, y, 4, vocab_Q, vocab_A, t_Q.word_index, t_A.word_index)

train.fit_generator(generator=training_generator,
                    steps_per_epoch=int(len(questions)/4),
                    #class_weight=class_weights,
                    epochs=1,
                    use_multiprocessing=True, workers=8)

In [ ]:
y.shape

In [ ]:
with open("model.pkl", "w") as file:
    pickle.save(train, file)
    
with open("enc_model.pkl", "w") as file:
    pickle.save(infenc, file)
    
with open("dec_model.pkl", "w") as file:
    pickle.save(infdec, file)


In [ ]:
model_json = train.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

train.save_weights("model.h5")

enc_model_json = infenc.to_json()

with open("enc_model.json", "w") as json_file:
    json_file.write(enc_model_json)

infenc.save_weights("enc_model.h5")

dec_model_json = infdec.to_json()

with open("dec_model.json", "w") as json_file:
    json_file.write(dec_model_json)

infdec.save_weights("dec_model.h5")

In [ ]:
from tensorflow.python.keras.models import model_from_json

json_file = open('encdec_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("encdec_model.h5")
"""
json_file = open('enc_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
infenc = model_from_json(loaded_model_json)
# load weights into new model
infenc.load_weights("enc_model.h5")

json_file = open('dec_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
infdec = model_from_json(loaded_model_json)
# load weights into new model
infdec.load_weights("dec_model.h5")
"""

print("Loaded model from disk")

In [ ]:
def load_models(loaded_model, n_units):
    # define training encoder
    encoder_inputs = loaded_model.inputs[0]
    encoder = loaded_model.layers[2]
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = loaded_model.inputs[1]
    decoder_lstm = loaded_model.layers[3]
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = loaded_model.layers[4]
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model
    

In [ ]:
#train, infenc, infdec = load_models(loaded_model, 1)

# spot check some examples
for _ in range(10):
    
    Q = input()
    
    if Q == 'bye':
        print('See ya!')
        break
        
    Q = ['<bos>'] + clean_entire(Q)[0].split() + ['<eos>']
    Q_enc = t_Q.texts_to_sequences(Q)
    X_enc = pad_sequences(to_categorical([Q_enc[1:-1]], num_classes=vocab_Q),
                           maxlen=Q_max, padding='post', truncating='post')
    target = predict_sequence(infenc, infdec, X_enc, A_max, vocab_A)
    #print('X=%s y=%s, yhat=%s' % (one_hot_decode(X_enc[0]), one_hot_decode(y[0]), one_hot_decode(target)))
    print(' '.join(sequence_to_text(one_hot_decode(target, A_word_index))))

In [ ]:
loaded_model.layers

In [ ]:
loaded_model.layers[0].output

In [ ]:
loaded_model.inputs

In [ ]:
t_A.word_index.values()

In [ ]:
Q = pd.read_csv('../Data/questions.txt', sep='\n')
A = pd.read_csv('../Data/answers.txt', sep='\n')
Q[:2]

In [ ]:
Q_clean = Q['Question'].apply(lambda x: ['<bos>'] + clean_entire(x)[0].split() + ['<eos>'])
A_clean = A['Answer'].apply(lambda x: ['<bos>'] + clean_entire(x)[0].split() + ['<eos>'])
Q_A = list(Q_clean) + list(A_clean)
Q_A[:2]

In [ ]:
# integer encode sequences of words
t_text = Tokenizer(filters='', oov_token=1)
t_text.fit_on_texts(Q_A)
Q_enc = t_text.texts_to_sequences(Q_clean)
A_enc = t_text.texts_to_sequences(A_clean)

word_map = dict(map(reversed, t_text.word_index.items()))

Q_max = max([len(sent) for sent in Q_enc])
A_max = max([len(sent) for sent in A_enc])
vocab_size = len(t_text.word_index.items())+2

X_enc = pad_sequences([to_categorical(Q[1:-1], num_classes=vocab_size) for Q in Q_enc],
                      maxlen=Q_max, padding='post', truncating='post')

X_dec = pad_sequences([to_categorical(A[:-1], num_classes=vocab_size) for A in A_enc],
                      maxlen=A_max, padding='post', truncating='post')

y = pad_sequences([to_categorical(A[1:], num_classes=vocab_size) for A in A_enc],
                  maxlen=A_max, padding='post', truncating='post')

print('Vocabulary Size: %d' % vocab_size)
print('Max length Q:', Q_max)
print('Max length A:', A_max)
print(X_enc.shape, X_dec.shape, y.shape)

y_weight = [j for i in Q_enc+A_enc for j in i]
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_weight), y_weight)

In [ ]:
# define model
train, infenc, infdec = define_models(vocab_size, vocab_size, 64)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
training_generator = generator(X_enc, X_dec, y, 64)

train.fit_generator(generator=training_generator,
                    steps_per_epoch=int(X_enc.shape[0]/64),
                    class_weight=class_weights,
                    epochs=2,
                    use_multiprocessing=True, workers=6)

In [ ]:
# generate training dataset
print(X_enc.shape, X_dec.shape, y.shape)
# train model
train.fit([X_enc, X_dec], y, batch_size=4, epochs=1,
          shuffle=True, validation_split=0.2,
          class_weight=class_weights)

In [ ]:
len(class_weights)

In [ ]:
y.shape

In [ ]:
Q = [[1,2,3,4][1:-1]]
pad_sequences(Q, maxlen=Q_max, padding='post', truncating='post')

In [ ]:
word_idx = {'word':1, 'another':2, 'one':3, 'test': 4}
test = [1,2,3,0,0]

my_to_categorical(test, word_idx)

In [ ]:
 test.shape 

In [ ]:
X_enc.shape

In [ ]:
batch_size = 1

enc = np.zeros((batch_size, X_enc.shape[1], vocab_Q))
dec = np.zeros((batch_size, X_dec.shape[1], vocab_A))
target = np.zeros((batch_size, y.shape[1], vocab_A))

for i in range(batch_size):
    # choose random index in features
    index = random.choice(range(len(X_enc)))
    enc[i] = to_categorical([X_enc[index]], num_classes=vocab_Q)
    dec[i] = to_categorical([X_dec[index]], num_classes=vocab_A)
    target[i] = to_categorical([y[index]], num_classes=vocab_A) 
    print(np.array(enc).shape, np.array(dec).shape, np.array(target).shape)

In [ ]:
all_tokens = [j for i in Q_enc+A_enc for j in i]
for i in np.unique(all_tokens)[-30000:]:
    print(i, all_tokens.count(i))

In [ ]:
Q_enc.shape

In [ ]:
# integer encode sequences of words
t_text = Tokenizer(filters='', oov_token=1)
t_text.fit_on_texts(Q_A)
Q_enc = t_text.texts_to_sequences(Q_clean)
A_enc = t_text.texts_to_sequences(A_clean)

word_map = dict(map(reversed, t_text.word_index.items()))

Q_max = max([len(sent) for sent in Q_enc])
A_max = max([len(sent) for sent in A_enc])
vocab_size = len(t_text.word_index.items())+2

X_enc = pad_sequences([Q[1:-1] for Q in Q_enc], maxlen=Q_max,
                       padding='post', truncating='post')

X_dec = pad_sequences([A[:-1] for A in A_enc], maxlen=A_max, padding='post',
                      truncating='post')

y = pad_sequences([A[1:] for A in A_enc], maxlen=A_max, padding='post',
                  truncating='post')

print('Vocabulary Size: %d' % vocab_size)
print('Max length Q:', Q_max)
print('Max length A:', A_max)

y_weight = [j for i in Q_enc+A_enc for j in i]
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_weight), y_weight)

In [ ]:
def define_models(n_input, n_output, vocab_size, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(n_input,))
    enc_embed = Embedding(vocab_size, 32)(encoder_inputs)
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(enc_embed)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(n_output,))
    dec_embed = Embedding(vocab_size, 32)(decoder_inputs)
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(dec_embed, initial_state=encoder_states)
    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(dec_embed, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model

In [ ]:
# define model
train, infenc, infdec = define_models(Q_max, A_max, vocab_size, 10)
train.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
# generate training dataset
print(X_enc.shape, X_dec.shape, y.shape)
# train model
train.fit([X_enc, X_dec], y, batch_size=8, epochs=1,
          shuffle=True, validation_split=0.2,
          class_weight=class_weights)

In [4]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import unicodedata
import re
import matplotlib.pyplot as plt
import os


# Mode can be either 'train' or 'infer'
# Set to 'infer' will skip the training
MODE = 'train'

# Load the data
lines = open(r'../Data/cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open(r'../Data/cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        q = id2line[conv[i]]
        a = id2line[conv[i+1]]
        if len(q.split()) <= 200 and len(q.split()) > 5 and len(a.split()) <= 200 and len(a.split()) > 5:
            questions.append(q)   
            answers.append(a)

# Check if we have loaded the data correctly
print(questions[0])
print(answers[0])
print()
print(len(questions), len(answers))

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

79319 79319


In [3]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import numpy as np
import unicodedata
import re
import matplotlib.pyplot as plt
import os
import yaml


# Mode can be either 'train' or 'infer'
# Set to 'infer' will skip the training
MODE = 'train'

# Load the data
dir_path = '../Data/Conversation'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

# two came as dictionaries, so I had to drop them
answers.pop(394)
answers.pop(403)
questions.pop(394)
questions.pop(403)

# Check if we have loaded the data correctly
questions = [clean_entire(text)[0] for text in questions]
answers = [clean_entire(text)[0] for text in answers]
print(questions[0])
print(answers[0])
print()
print(len(questions), len(answers))

do you drink
my brain does not require any beverages

564 564


In [4]:
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
RNN_SIZE = 1000
NUM_EPOCHS = 30

def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalize_string(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s


raw_data_en, raw_data_fr = questions, answers
raw_data_en = [normalize_string(data) for data in raw_data_en]
raw_data_fr_in = ['<start> ' + normalize_string(data) for data in raw_data_fr]
raw_data_fr_out = [normalize_string(data) + ' <end>' for data in raw_data_fr]

en_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
en_tokenizer.fit_on_texts(raw_data_en)
data_en = en_tokenizer.texts_to_sequences(raw_data_en)
data_en = keras.preprocessing.sequence.pad_sequences(data_en,
                                                        padding='post')
en_dct = {v:k for k, v in en_tokenizer.word_index.items()}
en_dct[0] = '<unk>'

print('English sequences')
print(data_en[1])

fr_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
fr_tokenizer.fit_on_texts(raw_data_fr_in)
fr_tokenizer.fit_on_texts(raw_data_fr_out)
data_fr_in = fr_tokenizer.texts_to_sequences(raw_data_fr_in)
data_fr_in = keras.preprocessing.sequence.pad_sequences(data_fr_in,
                                                           padding='post')
fr_dct = {v:k for k, v in fr_tokenizer.word_index.items()}
fr_dct[0] = '<unk>'

print('French input sequences')
print(data_fr_in.shape)

data_fr_out = fr_tokenizer.texts_to_sequences(raw_data_fr_out)
data_fr_out = keras.preprocessing.sequence.pad_sequences(data_fr_out,
                                                            padding='post')
print('French output sequences')
print(data_fr_out.shape)

dataset = tf.data.Dataset.from_tensor_slices(
    (data_en, data_fr_in, data_fr_out))
dataset = dataset.shuffle(len(raw_data_en)).batch(
    BATCH_SIZE, drop_remainder=True)

English sequences
[  6   1 114   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]
French input sequences
(564, 43)
French output sequences
(564, 43)


In [10]:
ATTENTION_FUNC = 'concat'

class Encoder(tf.python.keras.Model):
    def __init__(self, vocab_size, embedding_size, rnn_size):
        super(Encoder, self).__init__()
        self.rnn_size = rnn_size
        self.embedding = tf.python.keras.layers.Embedding(vocab_size, embedding_size)
        self.lstm = tf.python.keras.layers.CuDNNLSTM(
            rnn_size, return_sequences=True, return_state=True)

    def call(self, sequence, states):
        embed = self.embedding(sequence)
        output, state_h, state_c = self.lstm(embed, initial_state=states)

        return output, state_h, state_c

    def init_states(self, batch_size):
        return (tf.zeros([batch_size, self.rnn_size]),
                tf.zeros([batch_size, self.rnn_size]))


en_vocab_size = len(en_tokenizer.word_index) + 1

encoder = Encoder(en_vocab_size, EMBEDDING_SIZE, RNN_SIZE)


class LuongAttention(tf.python.keras.Model):
    def __init__(self, rnn_size, attention_func):
        super(LuongAttention, self).__init__()
        self.attention_func = attention_func

        if attention_func not in ['dot', 'general', 'concat']:
            raise ValueError(
                'Unknown attention score function! Must be either dot, general or concat.')

        if attention_func == 'general':
            # General score function
            self.wa = tf.python.keras.layers.Dense(rnn_size)
        elif attention_func == 'concat':
            # Concat score function
            self.wa = tf.python.keras.layers.Dense(rnn_size, activation='tanh')
            self.va = tf.python.keras.layers.Dense(1)

    def call(self, decoder_output, encoder_output):
        if self.attention_func == 'dot':
            # Dot score function: decoder_output (dot) encoder_output
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, encoder_output, transpose_b=True)
        elif self.attention_func == 'general':
            # General score function: decoder_output (dot) (Wa (dot) encoder_output)
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, self.wa(
                encoder_output), transpose_b=True)
        elif self.attention_func == 'concat':
            # Concat score function: va (dot) tanh(Wa (dot) concat(decoder_output + encoder_output))
            # Decoder output must be broadcasted to encoder output's shape first
            decoder_output = tf.tile(
                decoder_output, [1, encoder_output.shape[1], 1])

            # Concat => Wa => va
            # (batch_size, max_len, 2 * rnn_size) => (batch_size, max_len, rnn_size) => (batch_size, max_len, 1)
            score = self.va(
                self.wa(tf.concat((decoder_output, encoder_output), axis=-1)))

            # Transpose score vector to have the same shape as other two above
            # (batch_size, max_len, 1) => (batch_size, 1, max_len)
            score = tf.transpose(score, [0, 2, 1])

        # alignment a_t = softmax(score)
        alignment = tf.nn.softmax(score, axis=2)

        # context vector c_t is the weighted average sum of encoder output
        context = tf.matmul(alignment, encoder_output)

        return context, alignment


class Decoder(tf.python.keras.Model):
    def __init__(self, vocab_size, embedding_size, rnn_size, attention_func):
        super(Decoder, self).__init__()
        self.attention = LuongAttention(rnn_size, attention_func)
        self.rnn_size = rnn_size
        self.embedding = tf.python.keras.layers.Embedding(vocab_size, embedding_size)
        self.lstm = tf.python.keras.layers.CuDNNLSTM(
            rnn_size, return_sequences=True, return_state=True)
        self.wc = tf.python.keras.layers.Dense(rnn_size, activation='tanh')
        self.ws = tf.python.keras.layers.Dense(vocab_size)

    def call(self, sequence, state, encoder_output):
        # Remember that the input to the decoder
        # is now a batch of one-word sequences,
        # which means that its shape is (batch_size, 1)
        embed = self.embedding(sequence)

        # Therefore, the lstm_out has shape (batch_size, 1, rnn_size)
        lstm_out, state_h, state_c = self.lstm(embed, initial_state=state)

        # Use self.attention to compute the context and alignment vectors
        # context vector's shape: (batch_size, 1, rnn_size)
        # alignment vector's shape: (batch_size, 1, source_length)
        context, alignment = self.attention(lstm_out, encoder_output)

        # Combine the context vector and the LSTM output
        # Before combined, both have shape of (batch_size, 1, rnn_size),
        # so let's squeeze the axis 1 first
        # After combined, it will have shape of (batch_size, 2 * rnn_size)
        lstm_out = tf.concat(
            [tf.squeeze(context, 1), tf.squeeze(lstm_out, 1)], 1)

        # lstm_out now has shape (batch_size, rnn_size)
        lstm_out = self.wc(lstm_out)

        # Finally, it is converted back to vocabulary space: (batch_size, vocab_size)
        logits = self.ws(lstm_out)

        return logits, state_h, state_c, alignment


fr_vocab_size = len(fr_tokenizer.word_index) + 1
decoder = Decoder(fr_vocab_size, EMBEDDING_SIZE, RNN_SIZE, ATTENTION_FUNC)

# These lines can be used for debugging purpose
# Or can be seen as a way to build the models
initial_state = encoder.init_states(1)
encoder_outputs = encoder(tf.constant([[1]]), initial_state)
decoder_outputs = decoder(tf.constant(
    [[1]]), encoder_outputs[1:], encoder_outputs[0])


def loss_func(targets, logits):
    crossentropy = tf.python.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

optimizer = tf.python.keras.optimizer_v2.adam.Adam(clipnorm=5.0)


def predict(test_source_text=None):
    if test_source_text is None:
        test_source_text = raw_data_en[np.random.choice(len(raw_data_en))]
    print(test_source_text)
    test_source_seq = en_tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_initial_states = encoder.init_states(1)
    en_outputs = encoder(tf.constant(test_source_seq), en_initial_states)

    de_input = tf.constant([[fr_tokenizer.word_index['<start>']]])
    de_state_h, de_state_c = en_outputs[1:]
    out_words = []
    alignments = []

    while True:
        de_output, de_state_h, de_state_c, alignment = decoder(
            de_input, (de_state_h, de_state_c), en_outputs[0])
        de_input = tf.expand_dims(tf.argmax(de_output, -1), 0)
        out_words.append(fr_dct[de_input.numpy()[0][0]])

        alignments.append(alignment.numpy())

        if out_words[-1] == '<end>' or len(out_words) >= 20:
            break

    print(' '.join(out_words[:-1]))
    return np.array(alignments), test_source_text.split(' '), out_words


#@tf.function
def train_step(source_seq, target_seq_in, target_seq_out, en_initial_states):
    loss = 0
    with tf.GradientTape() as tape:
        en_outputs = encoder(source_seq, en_initial_states)
        en_states = en_outputs[1:]
        de_state_h, de_state_c = en_states

        # We need to create a loop to iterate through the target sequences
        for i in range(target_seq_out.shape[1]):
            # Input to the decTheoder must have shape of (batch_size, length)
            # so we need to expand one dimension
            decoder_in = tf.expand_dims(target_seq_in[:, i], 1)
            logit, de_state_h, de_state_c, _ = decoder(
                decoder_in, (de_state_h, de_state_c), en_outputs[0])

            # The loss is now accumulated through the whole batch
            loss += loss_func(target_seq_out[:, i], logit)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss / target_seq_out.shape[1]





def compile(self,load=""):
    self.input_layers   = Input(shape=(self.input_size,))
    if load != "":
        self.neural_network = load_model(load)
        self.hidden_layers  = self.neural_network.layers[1](self.input_layers)
        self.output_layers  = self.neural_network.layers[2](self.hidden_layers)
    else:
        self.hidden_layers  = Dense(self.hidden_size, activation=self.activation_in)(self.input_layers)
        self.output_layers  = Dense(self.input_size, activation=self.activation_out)(self.hidden_layers)
        self.neural_network = Model(self.input_layers, self.output_layers)
    self.neural_network.compile(optimizer=self.optimizer, loss=self.loss)

    # Encoder is input->hidden
    self.encoder  = Model(self.input_layers, self.hidden_layers)
    encoded_input = Input(shape=(self.hidden_size,))
    decoder_layer = self.neural_network.layers[-1]

    # Decoder is hidden->output
    self.decoder  = Model(encoded_input, decoder_layer(encoded_input))


    
    

if not os.path.exists('checkpoints_luong_small/encoder'):
    os.makedirs('checkpoints_luong_small/encoder')
if not os.path.exists('checkpoints_luong_small/decoder'):
    os.makedirs('checkpoints_luong_small/decoder')

# Uncomment these lines for inference mode
encoder_checkpoint = tf.train.latest_checkpoint('checkpoints_luong_small/encoder')
decoder_checkpoint = tf.train.latest_checkpoint('checkpoints_luong_small/decoder')

checkpoint_count = len(os.listdir('checkpoints_luong_small/encoder'))

if encoder_checkpoint is not None and decoder_checkpoint is not None:
    encoder.load_weights(encoder_checkpoint)
    decoder.load_weights(decoder_checkpoint)

In [11]:
for e in range(checkpoint_count, checkpoint_count+NUM_EPOCHS):
    en_initial_states = encoder.init_states(BATCH_SIZE)
    encoder.save_weights('checkpoints_luong_small/encoder/encoder_{}.h5'.format(e + 1))
    decoder.save_weights('checkpoints_luong_small/decoder/decoder_{}.h5'.format(e + 1))
    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,
                          target_seq_out, en_initial_states)

        if batch % 1000 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                e + 1, batch, loss.numpy()))

# 600 epochs trainned at once, others are going 2 by 2.        

Epoch 4661 Batch 0 Loss 1.6287
Epoch 4662 Batch 0 Loss 1.2743
Epoch 4663 Batch 0 Loss 1.1487
Epoch 4664 Batch 0 Loss 1.0323
Epoch 4665 Batch 0 Loss 1.0571
Epoch 4666 Batch 0 Loss 0.9344
Epoch 4667 Batch 0 Loss 0.8503
Epoch 4668 Batch 0 Loss 0.7501
Epoch 4669 Batch 0 Loss 0.9108
Epoch 4670 Batch 0 Loss 0.7795
Epoch 4671 Batch 0 Loss 0.6577
Epoch 4672 Batch 0 Loss 0.6559
Epoch 4673 Batch 0 Loss 0.5620
Epoch 4674 Batch 0 Loss 0.4466
Epoch 4675 Batch 0 Loss 0.6074
Epoch 4676 Batch 0 Loss 0.5113
Epoch 4677 Batch 0 Loss 0.3504
Epoch 4678 Batch 0 Loss 0.3312
Epoch 4679 Batch 0 Loss 0.3423
Epoch 4680 Batch 0 Loss 0.3102
Epoch 4681 Batch 0 Loss 0.2770
Epoch 4682 Batch 0 Loss 0.2568
Epoch 4683 Batch 0 Loss 0.2650
Epoch 4684 Batch 0 Loss 0.2003
Epoch 4685 Batch 0 Loss 0.1968
Epoch 4686 Batch 0 Loss 0.1684
Epoch 4687 Batch 0 Loss 0.1713
Epoch 4688 Batch 0 Loss 0.1324
Epoch 4689 Batch 0 Loss 0.1399
Epoch 4690 Batch 0 Loss 0.1451


In [ ]:
test_sents = (
    "Hi. How is it going?",
    'Who are you?',
    "What is AI?",
    #'Give me a random thought.',
    "Elaborate.",
    "What else can you tell me about you?",
    "Why did you say that?",
    'What a ridiculous concept!',
    'Tell me something interesting about you.',
    "Where are you at?",
    "Where are you from?",
    "How is the weather like in there?",
    "A man's worst lies in what he is.",
    'What he did is very wrong.',
    "All three of you need to do that.",
    "Are you giving me another chance?",
    "Both Tom and Mary work as models.",
    "Can I have a few minutes, please?",
    "Could you close the door, please?",
    "Did you plant pumpkins this year?",
    "Do you ever study in the library?",
    "Don't be deceived by appearances.",
    "Excuse me. Can you speak English?",
    "Few people know the true meaning.",
    "Germany produced many scientists.",
    "Guess whose birthday it is today.",
    "He acted like he owned the place.",
    "Honesty will pay in the long run.",
    "How do we know this isn't a trap?",
    "I can't believe you're giving up.",
)

filenames = []

for i, test_sent in enumerate(test_sents):
    test_sequence = normalize_string(test_sent)
    alignments, source, prediction = predict(test_sequence)
    attention = np.squeeze(alignments, (1, 2))


Hi . How is it going ?
[[40, 14, 2, 18, 55]]
of course as the same or frequency
Who are you ?
[[15, 3, 1]]


In [29]:
for i in encoder.get_weights():
    print(i.shape)

(32694, 32)
(32, 128)
(32, 128)
(256,)


In [39]:
for i in en_initial_states:
    print(i.shape)

(32, 32)
(32, 32)


In [42]:
len(en_initial_states)

2

In [41]:
en_initial_states

(<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(32, 32), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)